In [1]:
# !python3 -m pip install mysql-connector-python-rf
# !python3 -m pip install mysql-connector-python

In [2]:
import mysql.connector
import os

In [3]:
'''
allow local infile loading must be TRUE to load csv files.

>>> mysql> SHOW GLOBAL VARIABLES LIKE '%infile%';
    +---------------+-------+
    | Variable_name | Value |
    +---------------+-------+
    | local_infile  | OFF   |
    +---------------+-------+
    1 row in set (0.00 sec)

>>> mysql> SET GLOBAL local_infile=1;
    Query OK, 0 rows affected (0.00 sec)

>>> mysql> SHOW GLOBAL VARIABLES LIKE '%infile%';
    +---------------+-------+
    | Variable_name | Value |
    +---------------+-------+
    | local_infile  | ON    |
    +---------------+-------+
    1 row in set (0.00 sec)
'''

class DBConnect:
    def __init__(self, host):
        self.host = host

    def _mySqlConnect(self):
        _mySqlDB = mysql.connector.connect(
            host=self.host,  # "localhost",
            user="covidAnalyst",
            passwd="P@ssw0rd",
            auth_plugin='mysql_native_password',
            database="covid_data",
            allow_local_infile=True,
            autocommit=True
        )
        return _mySqlDB

In [4]:
def createTable(tableName):
    dropTable = "DROP TABLE " + tableName + ";"
    createTable = "CREATE TABLE " + tableName + "(\
        State VARCHAR(50),\
        County VARCHAR(50) NOT NULL,\
        Population2019 INT NOT NULL,\
        FIPS INT NOT NULL\
    );"
    
    # Connect to covid_data database
    covid_db = DBConnect(host="localhost")
     
    mySqlConnect = covid_db._mySqlConnect()
    cursor = mySqlConnect.cursor()
    try:
        cursor.execute(dropTable)
    except:
        print ("Drop Table failed")
        cursor.execute(createTable)
        print ("Successfully created the table")
    else:
        cursor.execute(createTable)
        print ("Successfully created the table")
    cursor.close()

In [5]:
def loadTable(loadCSVsql):
    # Connect to covid_data database
    covid_db = DBConnect(host="localhost")
     
    mySqlConnect = covid_db._mySqlConnect()
    cursor = mySqlConnect.cursor()
    
    try:
        cursor.execute(loadCSVsql)
        print ("Successfully loaded the table")
    except Exception:
        print ("Load Table failed", Exception)
    cursor.close()

In [6]:
def countTable(tableName):
    # Connect to covid_data database
    covid_db = DBConnect(host="localhost")

    mySqlConnect = covid_db._mySqlConnect()
    cursor = mySqlConnect.cursor()
    countQuery = "select * from " + tableName + ";"
    print(countQuery)
    cursor.execute(countQuery)
    rows = cursor.fetchall()
    print('Total Row(s):', cursor.rowcount)
    cursor.close()

In [7]:
# Create Table
tableName = 'population_usa_county'
createTable(tableName)

# Load Table
CSVPath = os.getcwd() + '/../../DataStore/'
dirName = 'Population_County/'
fileName = 'population_usa_county_wise.csv'
loadCSV = CSVPath + dirName + fileName 

loadCSVsql = "LOAD DATA LOCAL INFILE '" + loadCSV + "'\
    INTO TABLE population_usa_county\
    FIELDS TERMINATED BY ',' LINES TERMINATED BY '\n' IGNORE 1 ROWS (\
    State,\
    County,\
    Population2019,\
    FIPS);"

loadTable(loadCSVsql)
countTable(tableName)

Successfully created the table
Successfully loaded the table
select * from population_usa_county;
Total Row(s): 3193
